In [52]:
! git clone https://github.com/herrouzsalah/text_summarization_project.git

Cloning into 'text_summarization_project'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 30 (delta 10), reused 24 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 36.47 KiB | 5.21 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [53]:
%cd text_summarization_project/

/content/text_summarization_project/text_summarization_project


In [54]:
!pip install arabert

In [55]:
from transformers import GPT2TokenizerFast, pipeline
from transformers import GPT2LMHeadModel
from arabert.aragpt2.grover.modeling_gpt2 import GPT2LMHeadModel
from arabert.preprocess import ArabertPreprocessor

In [56]:
#TODO: Complete this cell
MODEL_NAME = "aubmindlab/aragpt2-base"
arabert_prep = ArabertPreprocessor(model_name=MODEL_NAME)

text="الجزائر بلد"
text_clean = arabert_prep.preprocess(text)

model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)
tokenizer = GPT2TokenizerFast.from_pretrained(MODEL_NAME)
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

#feel free to try different decoding settings
generation_pipeline(text,
    pad_token_id=tokenizer.eos_token_id,
    num_beams=10,
    max_length=200,
    top_p=0.9,
    repetition_penalty = 3.0,
    no_repeat_ngram_size = 3)[0]['generated_text']

Some weights of the model checkpoint at aubmindlab/aragpt2-base were not used when initializing GPT2LMHeadModel: ['ln_f.bias', 'ln_f.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at aubmindlab/aragpt2-base and are newly initialized: ['emb_norm.bias', 'emb_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_t

'الجزائر بلد ،جب ان هي انها فيه انه بصورة الأسرةرم في جميع الى أنها أنون الممثلة بشكل هى و كافة ولا شبه الأصل عن علىن نصفبنأن شكل النظام عبر الاسرةالات هو تعتبر بأن العلامةإالإ أيضا القيمة الأم نظ ى والاسرة فهي إلى المستقبل فى الافراد والمدر وتلك وهو وكلذا ممثلةلت ايضا الأفراد نور هما الشكل شكله العائلةحيات وكذلك وهي يعتبر الاصل الملاكالا�ث ون جيهان عليه 27 مثل بأكمله كبير من كيت وتعتبر 7نت مجموع الدولة له وبقية كلهم 11الت وباقي حتى تماما ويتم الشعب الملك وج 2ش آ 23 2017 منذ ج كلهاتى نورا كما الله ال ربي لجميع مست لة نظاموا أنهالل� القطاع كثيرة البياناالس بالفعل كبيرةتي مريم مختلف نتمنى اوج البندالس علية علاء. تلك والو 3 31 عمر بالتأكيد وعله دائما اليينئين وعلىالعه ضوء تبارك السيدةانته والدكتورة عموما مجتمعة معينينا شعب يب الل لذلك 1 بواسطة وبعض نوفبأ دليلاات لمختلف وشبه عبد بعض ماوين بالاضافة لغز فيصل الو بما نه ء انهماأت است بالتالي الاطفال الامة 26 لكل يعكس'

In [57]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(64000, 768)
    (wpe): Embedding(1024, 768)
    (emb_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
  )
  (lm_head): Linear(in_features=768, out_features=64000, bias=False)
)


In [115]:
from src.utils_data import *
from src.utils_tokenizer import *
from src.train import *

In [202]:
max_length = 512
sum_length = 150
split_probability = 0.9

In [203]:
train, val, test = process_data("/content/text_summarization_project/data/arabic_texts_summaries.csv",max_length , sum_length, split_probability)


train size: 5
val size: 22
test size: 23
test head:
                                                 text  \
29  تدور أحداث هذا النص حول مغامرة في الجبال. يبدأ...   

                                  summary  text_len  
29  تجربة تسلق جبال شاهقة واكتشاف الطبيعة        49  


In [204]:
# Add token to AraGPT2 tokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('aubmindlab/aragpt2-base')

special_tokens = {
    'bos_token': '<BOS>',  # Beginning of Sentence token
    'eos_token': '<EOS>',  # End of Sentence token
    'pad_token': '<PAD>',  # Padding token
    'additional_special_tokens': ['<SEP>']  # Separator token, if needed
}
tokenizer.add_special_tokens(special_tokens)

print('tokenizer len: {}'.format(len(tokenizer)))

ignore_idx = tokenizer.pad_token_id


tokenizer len: 64004


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [205]:
print("Special tokens added:")
print("BOS token ID:", tokenizer.convert_tokens_to_ids('<BOS>'))
print("EOS token ID:", tokenizer.convert_tokens_to_ids('<EOS>'))
print("PAD token ID:", tokenizer.convert_tokens_to_ids('<PAD>'))
print("SEP token ID:", tokenizer.convert_tokens_to_ids('<SEP>'))

Special tokens added:
BOS token ID: 64000
EOS token ID: 64001
PAD token ID: 64002
SEP token ID: 64003


In [206]:
# TODO: apply tokenizer
import os

tokenizer_dir ="tokenizer_path_save"
if not os.path.exists(tokenizer_dir):
  os.makedirs(tokenizer_dir) # Create output directory if needed

max_seq_len = 768
tokenizer.save_pretrained(tokenizer_dir)
tokenizer_len = len(tokenizer)
print('ignore_index: {}'.format(tokenizer.pad_token_id))
print('max_len: {}'.format(max_seq_len))

train, val, test = tokenize_dataset(tokenizer, train, val, test, max_seq_len)# Fix tokenize_dataset function in utils_tokenizer and call it


ignore_index: 64002
max_len: 768
Tokenized DataFrame head:
                                                 text  \
9   تدور أحداث هذا النص حول تجربة في مطعم جديد. يب...   
15  تدور أحداث هذا النص حول رحلة في الصحراء. يبدأ ...   
6   تدور أحداث هذا النص حول حفلة في القرية. يبدأ ا...   
25  تدور أحداث هذا النص حول يوم دراسي. يبدأ النص ب...   
8   تدور أحداث هذا النص حول قصة حب في المدينة. يبد...   

                                              summary  text_len  \
9                تذوق أطباق عالمية في مطعم جديد وأنيق        52   
15  رحلة مثيرة في الصحراء تكشف جمال الطبيعة الصحراوية        49   
6   حفلة تراثية في قرية تظهر العادات والتقاليد الم...        49   
25               يوم مليء بالتعلم والتفاعل في المدرسة        46   
8        قصة حب مؤثرة تدور أحداثها في المدينة الصاخبة        52   

                      encodings  
9   [input_ids, attention_mask]  
15  [input_ids, attention_mask]  
6   [input_ids, attention_mask]  
25  [input_ids, attention_mask]  
8   [input_ids, attention

In [207]:
#Generate train/val/test files
#save tokenized data
out_dir="tokenizer_data"
processed_set= "dataset"
data_dir = os.path.join(out_dir, processed_set)
if not os.path.exists(data_dir):
  os.makedirs(data_dir) # Create output directory if needed
train_file = os.path.join(data_dir,"train.csv")
train.to_csv(train_file, index=False)

val_file = os.path.join(data_dir,"val.csv")
val.to_csv(val_file, index=False)

test_file = os.path.join(data_dir,"test.csv")
test.to_csv(test_file, index=False)


In [65]:
# TODO: Visualize train and explain each column

In [208]:
train_file = os.path.join(data_dir, "train.csv")
train_df = pd.read_csv(train_file)
print(train_df.head())

   text_len                                          encodings
0        52  {'input_ids': tensor([[64000,  8908,  5368,   ...
1        49  {'input_ids': tensor([[64000,  8908,  5368,   ...
2        49  {'input_ids': tensor([[64000,  8908,  5368,   ...
3        46  {'input_ids': tensor([[64000,  8908,  5368,   ...
4        52  {'input_ids': tensor([[64000,  8908,  5368,   ...


In [209]:
# TODO: Data Loaders
# Fix code in utils_data.py
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import pandas as pd

train_dataset, val_dataset = get_gpt2_dataset(train, val)		# call function get_gpt2_dataset
#b = train_dataset.__getitem__(1) # check one data row
#print(b)
print(train_dataset.sum_idx[2])
train_dataloader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size = 1)
val_dataloader = DataLoader(val_dataset, sampler = SequentialSampler(val_dataset), batch_size = 1)

train_loader_len = len(train_dataloader)
print('Number of batches in training data loader:', train_loader_len)

49
Number of batches in training data loader: 5


In [210]:
config = {
    "out_dir": "/content/text_summarization_project/models",
    "training_models": "gpt2_model",
    "final_model": "gpt2_finetuned_final",
    "learning_rate": 5e-5,
    #"num_train_epochs": 3,
    #"batch_size": 16
}

# fine tune pretrained model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_dir =  'aubmindlab/aragpt2-base'

train = Train(device, model_dir, tokenizer_len, ignore_idx, train_loader_len, config)
train.train_model(train_dataloader, val_dataloader)



======== Epoch 1/50 ========
Batch 0: {'text': tensor([[[64000,  8908,  5368,   542,  3499,  1027,  6205,   305,  7427,    17,
           5688,  3499, 12594, 22409, 39915, 20162,  1366, 10279,   152,   238,
            952, 17073,  1464,   879,  5127, 34742, 22933,   468, 24025,  4777,
          15437,   305,  6205,   305,  7427,    17, 12531,  2235,  6452,  6258,
            599, 13204, 41148, 12213,  3600, 22610, 10768,   534,   226, 13456,
            305,  7427,   152,   238,  1587,  4801,  8044,  6618, 13064,  3151,
            308,  6452,    17,   224, 64003,  6205, 13798,   305,  7427,  8654,
           3655,  7241, 22810,   224, 64001, 64002, 64002, 64002, 64002, 64002,
          64002, 64002, 64002, 64002, 64002, 64002, 64002, 64002, 64002, 64002,
          64002, 64002, 64002, 64002, 64002, 64002, 64002, 64002, 64002, 64002,
          64002, 64002, 64002, 64002, 64002, 64002, 64002, 64002, 64002, 64002,
          64002, 64002, 64002, 64002, 64002, 64002, 64002, 64002, 64002, 

<ipython-input-200-4b3153aaeb7d>:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  text = torch.tensor(encoding['input_ids'], dtype=torch.long)  # Extract input_ids from encodings
<ipython-input-200-4b3153aaeb7d>:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attn_mask = torch.tensor(encoding['attention_mask'], dtype=torch.long)  # Extract attention_mask from encodings


ValueError: Expected input batch_size (768) to match target batch_size (767).

In [215]:
import torch
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import GPT2LMHeadModel, GPT2Config, AdamW, get_linear_schedule_with_warmup
from torch.nn import CrossEntropyLoss
import os
import time
from torch.optim import AdamW

class Train(object):
    def __init__(self, device, model_path, tokenizer_len, ignore_index, train_loader_len, config):
        self.device = device
        self.config = config
        # Define paths for saving models
        self.final_model = os.path.join(config['out_dir'], config['final_model'])
        if not os.path.exists(self.final_model):
            os.makedirs(self.final_model)

        self.training_models = os.path.join(config['out_dir'], config['training_models'])
        if not os.path.exists(self.training_models):
            os.makedirs(self.training_models)


        # Load model and tokenizer
        self.configuration = GPT2Config.from_pretrained(model_path, output_hidden_states=False)
        self.model = GPT2LMHeadModel.from_pretrained(model_path, config=self.configuration)
        self.model.resize_token_embeddings(tokenizer_len)
        self.model.to(self.device)

        # Initialize optimizer and loss function
        self.optimizer = AdamW(self.model.parameters(), lr=self.config.get('learning_rate', 5e-5))
        #self.optimizer = AdamW(self.model.parameters(), lr=5e-4, eps=1e-8)
        self.loss_fct = CrossEntropyLoss(ignore_index=ignore_index)  # Ignores padding token for loss calculation

        self.gradient_accumulation_steps = 32
        self.max_grad_norm = 1

        # Create the learning rate scheduler
        self.scheduler = get_linear_schedule_with_warmup(
            self.optimizer,
            num_warmup_steps=int(1e2),
            num_training_steps=train_loader_len * 50
        )

    def process_train_batch(self, batch):
        inputs, labels = batch['text'].to(self.device), batch['text'].to(self.device)
        logits = self.model(input_ids=inputs)[0]

        # Shift logits and labels to only consider the loss on the summary
        shift_logits = logits[..., 1:].contiguous()
        shift_labels = labels[..., 1:].contiguous()

        return shift_logits, shift_labels

    def process_eval_batch(self, batch):
        inputs, labels = batch['text'].to(self.device), batch['text'].to(self.device)
        with torch.no_grad():
            logits = self.model(input_ids=inputs)[0]

            # Shift logits and labels to only consider the loss on the summary
            shift_logits = logits[..., 1:].contiguous()
            shift_labels = labels[..., 1:].contiguous()

        return shift_logits, shift_labels
    def train_loop(self, train_dataloader):
        self.model.train()
        total_train_loss = 0

        for step, batch in enumerate(train_dataloader):
            print(f"Batch {step}: {batch}")  # Debug print

            self.model.zero_grad()  # put all gradients to zero
            shift_logits, shift_labels = self.process_train_batch(batch)

            loss = self.loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            loss = loss / self.gradient_accumulation_steps
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.max_grad_norm)
            self.optimizer.step()
            self.scheduler.step()
            batch_loss = loss.item()
            total_train_loss += batch_loss

        return total_train_loss



    def eval_loop(self, val_dataloader):
        self.model.eval()
        total_eval_loss = 0

        for step, batch in enumerate(val_dataloader):
            shift_logits, shift_labels = self.process_eval_batch(batch)
            loss = self.loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            batch_loss = loss.item()
            total_eval_loss += batch_loss

        return total_eval_loss

    def average_loss(self, total_loss, size):
        return total_loss / size

    def save_model(self, path):
        model_to_save = self.model.module if hasattr(self.model, 'module') else self.model
        model_to_save.save_pretrained(path)

    def model_params(self):
        name = "params_" + str(int(time.time())) + ".txt"
        file = os.path.join(self.config.out_dir, name)
        params = list(self.model.named_parameters())
        with open(file, "w") as f:
            f.write('The model has {:} different named parameters.\n'.format(len(params)))
            f.write('\n==== Embedding Layer ====\n')
            for p in params[0:2]:
                f.write("{:<55} {:>12}\n".format(p[0], str(tuple(p[1].size()))))

            f.write('\n==== First Transformer ====\n')
            for p in params[2:14]:
                f.write("{:<55} {:>12}\n".format(p[0], str(tuple(p[1].size()))))

            f.write('\n==== Output Layer ====\n')
            for p in params[-2:]:
                f.write("{:<55} {:>12}\n".format(p[0], str(tuple(p[1].size()))))

    def train_model(self, train_dataloader, val_dataloader):
        epochs = 50
        for epoch_i in range(epochs):
            print(f'======== Epoch {epoch_i + 1}/{epochs} ========')

            total_train_loss = self.train_loop(train_dataloader)
            avg_train_loss = self.average_loss(total_train_loss, len(train_dataloader))
            print(f"  Average training loss: {avg_train_loss:.2f}")

            total_eval_loss = self.eval_loop(val_dataloader)
            avg_val_loss = self.average_loss(total_eval_loss, len(val_dataloader))
            print(f"  Valid. Loss: {avg_val_loss:.2f}")

            if (epoch_i + 1) % 10 == 0:
                print(f'Saving model for epoch {epoch_i + 1}')
                path = os.path.join(self.training_models, str(epoch_i + 1))
                self.save_model(path)

        print("Training complete!")
        self.save_model(self.final_model)


In [216]:

# Utility functions for data manipulation and preparation

import pandas as pd
import numpy as np
import random

from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
from torchvision import transforms, utils
def tokenize_text(tokenizer, df, max_len):
    df['encodings'] = df.apply(lambda x: apply_tokenizer(tokenizer, x['text'], x['summary'], max_len), axis=1)
    print(f"Tokenized DataFrame head:\n{df.head()}")  # Debug print

    del df['text']
    del df['summary']

    return df

# Dataset definition
class GPT2Dataset(Dataset):
    def __init__(self, df):
        self.encodings = df['encodings'].to_list()
        self.sum_idx = df['text_len'].to_list()

    def __len__(self):
        return len(self.sum_idx)

    def __getitem__(self, idx):
        encoding = self.encodings[idx]
        if 'input_ids' not in encoding or 'attention_mask' not in encoding:
            raise KeyError("Encoding missing required keys: 'input_ids' or 'attention_mask'")

        # Convert lists to tensors
        text = torch.tensor(encoding['input_ids'], dtype=torch.long)  # Extract input_ids from encodings
        attn_mask = torch.tensor(encoding['attention_mask'], dtype=torch.long)  # Extract attention_mask from encodings
        s_idx = self.sum_idx[idx] + 2  # Adjust index for BOS and EOS tokens

        return {'text': text, 'mask': attn_mask, 's_idx': s_idx}


def get_gpt2_dataset(train, val):

  train_dataset = GPT2Dataset(train)
  val_dataset = GPT2Dataset(val)

  return train_dataset, val_dataset


def short_text(text, max_len):
	text = text.split()
	#print(len(text))
	#len= len-1
	s_text = text[:max_len]
	s_text = ' '.join(s_text)
	return  s_text


def process_dataframe(df, max_text, max_sum):

	df['text'] = df['text'].apply(lambda x: short_text(x, max_text))
	df['summary'] = df['summary'].apply(lambda x: short_text(x, max_sum))
	df['text_len'] = df['text'].apply(lambda x: len(x.split()))
	#print(df['summary'].str.split().str.len())
	#df['summary'] = df['summary'].apply(lambda x: ' <CLS> ' + x) # do this step in data loader
	#df['ts'] = df[['text', 'summary']].apply(lambda x: ''.join(x), axis=1)
	#print(df['ts'].str.split().str.len())

	return df

def split_data (df, sr):
	train, val_test = train_test_split(df,test_size=sr)
	val, test = train_test_split(val_test,test_size=0.5)
	return train, val, test

def process_data(file, max_text, max_sum, sr):

	# load into a data frame
	df = pd.read_csv(file)
	df = process_dataframe(df, max_text, max_sum)
	train, val, test = split_data(df, sr)
	print('train size: {}'.format(len(train)))
	print('val size: {}'.format(len(val)))
	print('test size: {}'.format(len(test)))
	print('test head:\n{}'.format(test.head(1)))

	return train, val, test


In [217]:
# Utility functions for tokenization
def apply_tokenizer(tokenizer, text, summary, max_len):
	# TODO: Complete this function
	# Concatenate text and summary and in-between add a special token
	# <BOS> is beginning of sentence
	# <EOS> is end of sentence
  combined_text = f'<BOS> {text} <SEP> {summary} <EOS>'
  encoding = tokenizer(combined_text,
                         truncation=True,
                         max_length=max_len,
                         padding="max_length",
                         return_tensors='pt')

  return encoding
	#return tokenizer('<BOS>'  '<EOS>', truncation=True, max_length=max_len, padding="max_length")

def tokenize_text(tokenizer, df, max_len):
    df['encodings'] = df.apply(lambda x : apply_tokenizer(tokenizer, x['text'], x['summary'], max_len), axis=1)
    #encodings is dict type: contains input_ids and attention_mas
    df['input_ids'] = df['encodings'].apply(lambda x: x['input_ids'].squeeze().tolist())
    df['attention_mask'] = df['encodings'].apply(lambda x: x['attention_mask'].squeeze().tolist())

    del df['text']
    del df['summary']
    del df['encodings']

    return df

def tokenize_dataset(tokenizer, train, val, test, max_len):
	train = tokenize_text(tokenizer, train, max_len)
	val = tokenize_text(tokenizer, val, max_len)
	test = tokenize_text(tokenizer, test, max_len)
	return train, val, test